## Тестирование Mongodb

In [22]:
import os

os.system('docker run -d -p 27017:27017 --name mongo mongo:latest')

4541fc40aaebb7a7da4db827f3b531a122b0d754ab44398f619dc4f2067f4b64


0

### Подключаемся к БД

In [23]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [24]:
from pymongo import MongoClient, ASCENDING
import uuid
import random

client = MongoClient('localhost', 27017)
db = client.test_database
collection = db.rating

## Тесты

In [25]:
data = []
movie_uuids = [str(uuid.uuid4()) for _ in range(1000)]
user_uuids = [str(uuid.uuid4()) for _ in range(1000)]
for _ in range(1_000_000):
    test_data = {
        "movie_uuid": random.choice(movie_uuids),
        "user_uuid": random.choice(user_uuids),
        "rating": random.randint(0, 10)
    }
    data.append(test_data)
collection.insert_many(data)
collection.create_index([('movie_uuid', ASCENDING)])

'movie_uuid_1'

In [36]:
%%time
pipeline = [
    {
        "$match": {"movie_uuid": movie_uuids[0]}
    },
    {
        "$project": {
            "movie_uuid": 1,
            "rating": 1,
            "equal0": {
                "$cond": [{"$eq": ["$rating", 0]}, 1, 0]
            },
            "equal10": {
                "$cond": [{"$eq": ["$rating", 10]}, 1, 0]
            }
        }
    },
    {
        "$group": {
            "_id": "$movie_uuid",
            "like": {"$sum": "$equal10"},
            "dislike": {"$sum": "$equal0"},
            "rating": {"$avg": "$rating"}
        }
    }
]
list(collection.aggregate(pipeline))

CPU times: user 1.05 ms, sys: 1.02 ms, total: 2.07 ms
Wall time: 15.5 ms


[{'_id': 'b315ed65-bbc9-4415-b2ad-e70e24d905c9',
  'like': 106,
  'dislike': 82,
  'rating': 5.127906976744186}]

In [37]:
%%time
one_doc = {
    "movie_uuid": movie_uuids[0],
    "user_uuid": user_uuids[0],
    "rating": random.randint(0, 10)
}
collection.insert_one(one_doc)

CPU times: user 1.02 ms, sys: 1.45 ms, total: 2.47 ms
Wall time: 11.8 ms
